In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import scipy.stats as st
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
data = pd.read_csv('time.txt', parse_dates=[0], dayfirst=True)

In [ ]:
groups = data['Time'].groupby(data['Time']).count()

In [ ]:
data = groups.cumsum()

In [ ]:
plt.plot(data)

STL:

In [ ]:

from statsmodels.tsa.seasonal import seasonal_decompose
stl_data = seasonal_decompose(data, freq=7)
_= stl_data.plot()

# ARIMA

In [ ]:
from statsmodels.tsa.stattools import kpss
kpss(data)

Почему так?

In [ ]:
new_data = {}
time_iter = pd.date_range('20190210','20190401',freq='D')
old_data = None
for i in time_iter:
    if i in data:
        new_data[i] = float(data[i])
        old_data = new_data[i] 
    else:
        new_data[i] = old_data

new_data = pd.DataFrame({'Time':new_data})


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
stl_data = seasonal_decompose(new_data, freq=7)
_= stl_data.plot()

In [ ]:
new_data_without_trend = (new_data- stl_data.trend)[3:-3]
plt.plot(new_data_without_trend)

In [ ]:
kpss(new_data['Time'])

In [ ]:
kpss(new_data_without_trend['Time'])

In [ ]:
plt.plot(new_data_without_trend.diff(7))

In [ ]:
kpss(new_data_without_trend['Time'].diff(7)[7:])

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
arima = ARIMA(new_data_without_trend, order=(1,1,1)).fit()

arima.summary()
plt.plot(new_data_without_trend, alpha=0.5, ls='--')
plt.plot(arima.predict(1, len(new_data_without_trend), dynamic=False))



In [ ]:
_ =plt.plot(arima.resid)
plt.show()
_ = plt.hist(arima.resid)
plt.show()
st.shapiro(arima.resid)

In [ ]:
arima = SARIMAX(new_data_without_trend, order=(1,0,1), seasonal_order=(0,2,1,7)).fit()

arima.summary()


In [ ]:
plt.plot(new_data_without_trend, alpha=0.5, ls='--')
plt.plot(arima.predict(0, len(new_data)))


In [ ]:
_ =plt.plot(arima.resid)
plt.show()
_ = plt.hist(arima.resid)
plt.show()
st.shapiro(arima.resid)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
plt.scatter(range(30), acorr_ljungbox(arima.resid, lags=30)[1])


In [ ]:
predictition = arima.get_prediction(0, len(new_data_without_trend)-1).conf_int()
intervals = stl_data.trend.values.flatten()[3:-3] + predictition['lower Time'],stl_data.trend.values.flatten()[3:-3] + predictition['upper Time']
plt.fill_between(intervals[0].index, intervals[0], intervals[1], alpha=0.2)
plt.ylim(0,250)
plt.plot(new_data, alpha=0.5, ls='--', c='r')
plt.plot(stl_data.trend.values.flatten()[3:-3]+arima.predict(0, len(new_data_without_trend)-1), c='g')


# ETS

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
ets = ExponentialSmoothing(new_data['Time'], trend='add', seasonal='add', seasonal_periods=7).fit()
plt.plot(new_data, alpha=0.5, ls='--', c='r')
predicted = ets.predict(0, 60)

plt.plot(predicted)